In [1]:
%display latex

In [3]:
N = 5
F = GF(2^N, 'x')
x = F.gen()

In [39]:
# How did I find this basis? Jungnickel's paper and brute force...
basis = [x^3, x^5, x^11, x^22, x^24]

In [41]:
def checkSelfDual(basis):
    for a in basis:
        for b in basis:
            if a == b:
                if (a * b).trace() != F(1):
                    raise Exception(a,b)
            else:
                if (a * b).trace() != F(0):
                    raise Exception(a,b)

In [42]:
checkSelfDual(basis)

In [43]:
# express an element as a linear combination of the basis
components = lambda k: [(k * el).trace() for el in basis]

# additive group character
chi = lambda k: exp(pi * I * int(k.trace()))

def toInt(k):
    return list(F).index(k)

In [44]:
y = var('y')

# compute the coefficients for the basis elements,
def solve_basis(basis, curve):
    sols = []
    for k in basis:
        sol = solve(chi(k * curve(k)) == y^2, y, solution_dict=True)
        sols.append(sol[1]) # positive solutions only (choice)
    return sols

In [45]:
# after computing the solutions we can simply
# get them for each basis element.
def basis_c(a, l, sols):
    if a == 0:
        return 1
    return sols[l][y]

In [46]:
# Compute an arbitrary coefficient c_{\alpha,f} for a 
# given curve f using the general formula.
def c(alpha, curve, sols=None):
    if not sols:
        sols = solve_basis(basis, curve)
    
    # Expand alpha in the basis
    comps = components(alpha)

    # Apply the formula
    s1 = 0
    for k in range(N-1):
        s2 = 0
        for j in range(k+1, N):
            s2 += comps[j] * basis[j]
        s1 += s2 * curve(comps[k] * basis[k])
        
    return chi(s1) * prod([basis_c(a, l, sols) for l, a in enumerate(comps)])

In [47]:
def sign_perm(sols, perm=None):
    if perm:
        for k, sol in enumerate(sols):
            sols[k][y] = perm[k] * sols[k][y]
    return sols

In [48]:
for mu in F:
    curve = lambda t: mu * t # \beta = \mu \alpha
    sols = sign_perm(solve_basis(basis, curve), [1,1,1,1,1])
    for k in F:
        for kp in F:
            lhs = c(k, curve, sols) * c(kp, curve, sols)
            rhs = chi(kp * curve(k)) * c(k + kp, curve, sols)
            if lhs != rhs:
                raise Exception('Recurrence relation does not hold!', mu)
print('Recurrence relation holds!')

Recurrence relation holds!


In [49]:
def ray(mu):
    return lambda t: mu * t

In [50]:
def PS(curve, perms):
    phase_space = zero_matrix(SR, 2^N, 2^N)
    phase_space[0,:] = 1 # vertical line
    for j, mu in enumerate(F): # iterate through the curve parametr
        # same sign choice for a fixed curve parameter
        sols = sign_perm(solve_basis(basis, curve(mu)), perms[j])
        for i, a in enumerate(F):
            # loop through alpha to obtain coefficient and
            # corresponding point
            coeff = c(a, curve(mu), sols)
            b = curve(mu)(a)
            phase_space[i, toInt(b)] = coeff
    return phase_space

In [52]:
ps = PS(ray, [[1,1,1,1,1]] * 32)
ps

32 x 32 dense matrix over Symbolic Ring (use the '.str()' method to see the entries)

Kantor's spread which is not isomorphic to the Desarguesian spread is given by the opperation:
$$
\alpha = 0,
\quad
\beta = m^2 \alpha + m T(\alpha) + T(m\alpha),
$$
for $m \in GF(2^5)$ and $T$ is the field trace function.

In [53]:
def Kantor(m):
    return lambda a: m^2 * a + m * a.trace() + (m * a).trace()

Let's verify that this operation produces an affine plane, i.e., each line is a subspace and there is no non-trivial intersection between lines.

In [58]:
def checkAdditivity(points):
    for a1, b1 in points:
        for a2, b2 in points:
            if (a1 + a2, b1 + b2) not in points:
                raise Exception('Not additive!')

In [59]:
for m in F:
    points = [(a, Kantor(m)(a)) for a in F]
    checkAdditivity(points)

In [60]:
for m1 in F:
    for m2 in F:
        if m1 != m2:
            curve1 = [(a, Kantor(m1)(a)) for a in F]
            curve2 = [(a, Kantor(m2)(a)) for a in F]
            if len(list(set(curve1) & set(curve2))) != 1:
                raise Exception('Found non-trivial intersection!')

So it is defined perfectly. Now let's test to see if any of the curves are abelian given our choice of phase space $\Gamma$.

In [61]:
def Proj(u, v=None):
    if not v:
        v = u
    return u.tensor_product(v.conjugate_transpose())

Id = identity_matrix(2^N)

def Fourier():
    s = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

In [62]:
def phi(a, b):
    return ps[toInt(a), toInt(b)]

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

def D(a, b):
    return phi(a, b) * Z(a) * X(b)

In [103]:
def testCurve(points):
    for p1 in points:
        for p2 in points:
            p_op = D(p1[0], p1[1]) @ D(p2[0], p2[1])
            s_op = D(p1[0] + p2[0], p1[1] + p2[1]) # assuming additivity
            if not np.allclose(p_op, s_op):
                raise Exception(p1, p2)
    return True

Takes way too long, time to bring `numpy`.

In [104]:
import numpy as np

In [105]:
chi = lambda k: np.exp(np.pi * 1j * int(k.trace()))

def Proj(u, v=None):
    if not v:
        v = u
    return np.outer(u, v.conj().T)

Id = np.eye(2^N)

def Fourier():
    s = np.zeros((2^N, 2^N), dtype='complex128')
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / np.sqrt(2^N)
    return s
FF = Fourier()

def phi(a, b):
    return complex(ps[toInt(a), toInt(b)])

def Z(a):
    return np.diag([chi(a * k) for k in F])

def X(b):
    return FF.conj().T @ Z(b) @ FF

def D(a, b):
    return phi(a, b) * Z(a) @ X(b)

In [106]:
points = [(a, Kantor(F(0))(a)) for a in F]
testCurve(points)

True

In [107]:
for m in F:
    print('Testing with m = {}'.format(m))
    points = [(a, Kantor(m)(a)) for a in F]
    try:
        testCurve(points)
        print(m, 'abelian')
    except:
        continue

Testing with m = 0
0 abelian
Testing with m = x
Testing with m = x^2
Testing with m = x^3
Testing with m = x^4
Testing with m = x^2 + 1
Testing with m = x^3 + x
Testing with m = x^4 + x^2
Testing with m = x^3 + x^2 + 1
Testing with m = x^4 + x^3 + x
Testing with m = x^4 + 1
Testing with m = x^2 + x + 1
Testing with m = x^3 + x^2 + x
Testing with m = x^4 + x^3 + x^2
Testing with m = x^4 + x^3 + x^2 + 1
Testing with m = x^4 + x^3 + x^2 + x + 1
Testing with m = x^4 + x^3 + x + 1
Testing with m = x^4 + x + 1
Testing with m = x + 1
Testing with m = x^2 + x
Testing with m = x^3 + x^2
Testing with m = x^4 + x^3
Testing with m = x^4 + x^2 + 1
Testing with m = x^3 + x^2 + x + 1
Testing with m = x^4 + x^3 + x^2 + x
Testing with m = x^4 + x^3 + 1
Testing with m = x^4 + x^2 + x + 1
Testing with m = x^3 + x + 1
Testing with m = x^4 + x^2 + x
Testing with m = x^3 + 1
Testing with m = x^4 + x
Testing with m = 1
1 abelian


No non-trivial abelian curves found! Does this have to do with the fact that the affine planes are not inequivalent?

In [108]:
testCurve([(a,Kantor(x)(a)) for a in F])

Exception: ((x, x^3), (x^4, x^3 + x + 1))